In [37]:
import json
import pandas as pd
import plotly.express as px
from geopy import Nominatim, distance

# Transporte público

## Trenes

In [38]:
df_stops = pd.read_csv("../data/external/transporte_publico/trenes/stops.txt")

In [39]:
df_stops["type"] = "tren"
df_stops = df_stops.rename(columns={"stop_lat": "lat", "stop_lon": "lon"})

In [40]:
df_stops.head()

,stop_id,stop_name,lat,lon,type
0,154,González Catán,-34.771464,-58.646546,tren
1,178,Independencia,-34.760798,-58.617341,tren
2,180,Ing. Budge,-34.718663,-58.462152,tren
3,184,Isidro Casanova,-34.704464,-58.587990,tren
4,203,Justo Villegas,-34.708965,-58.551939,tren


In [41]:
df_routes = pd.read_csv("../data/external/transporte_publico/trenes/routes.txt")
df_routes = df_routes[["route_id", "route_short_name", "route_long_name"]]
df_routes = df_routes.drop_duplicates(keep="last")

In [42]:
df_trips = pd.read_csv("../data/external/transporte_publico/trenes/trips.txt")
df_trips = df_trips[['route_id',  'trip_id']]
df_trips = df_trips.drop_duplicates(keep="last")

In [43]:
df_stop_times = pd.read_csv("../data/external/transporte_publico/trenes/stop_times.txt")
df_stop_times = df_stop_times[['trip_id', 'stop_id']]
df_stop_times = df_stop_times.drop_duplicates(keep="last")

In [44]:
df_tren = df_routes.merge(df_trips, on="route_id")
df_tren = df_tren.merge(df_stop_times, on="trip_id")
df_tren = df_tren.merge(df_stops, on="stop_id")

df_tren = df_tren.drop_duplicates(subset=["lat", "lon"], keep="first")

df_tren.head(2)

,route_id,route_short_name,route_long_name,trip_id,stop_id,stop_name,lat,lon,type
0,1,Sarmiento,Sarmiento / Once - Moreno,32001211,70,Castelar,-34.652156,-58.642526,tren
626,1,Sarmiento,Sarmiento / Once - Moreno,32001211,279,Morón,-34.648223,-58.619813,tren


In [45]:
df_tren = df_tren[["route_id", "route_short_name", "route_long_name", "stop_id", "stop_name", "lat", "lon", "type"]]
df_tren = df_tren.rename(columns={"route_short_name": "linea", "route_long_name": "ramal"})
df_tren["ramal"] = df_tren["ramal"].str.split("/")
df_tren["ramal"] = df_tren["ramal"].apply(lambda x: x[1])

df_tren.head(2)

,route_id,linea,ramal,stop_id,stop_name,lat,lon,type
0,1,Sarmiento,Once - Moreno,70,Castelar,-34.652156,-58.642526,tren
626,1,Sarmiento,Once - Moreno,279,Morón,-34.648223,-58.619813,tren


In [46]:
centro_geografico_caba = [-34.61448064637565, -58.44638197269814]
centro_geografico_plata = [-34.92137784728068, -57.95455274162061]
df_tren["dist_centro_caba"] = df_tren.apply(lambda x: distance.distance(centro_geografico_caba, [x.lat, x.lon]).km, axis=1)
df_tren["dist_centro_plata"] = df_tren.apply(lambda x: distance.distance(centro_geografico_plata, [x.lat, x.lon]).km, axis=1)

df_tren = df_tren.drop_duplicates(keep="last")

In [47]:
df_tren.to_parquet("../data/interim/transporte/trenes_all.parquet")

In [48]:
df_tren = df_tren[(df_tren.dist_centro_caba < 11) | (df_tren.dist_centro_plata < 17)]
df_tren = df_tren[["linea", "ramal", "stop_name", "lat", "lon", "type"]]

df_tren.to_parquet("../data/interim/transporte/trenes_important.parquet", index=False, engine="pyarrow")

In [49]:
df_tren.head(2)

,linea,ramal,stop_name,lat,lon,type
2455,Sarmiento,Once - Moreno,Ciudadela,-34.639923,-58.541123,tren
3051,Sarmiento,Once - Moreno,Liniers,-34.638873,-58.526341,tren


## Subte

In [50]:
df_subte = pd.read_csv("../data/external/transporte_publico/subte/estaciones-de-subte.csv")

In [51]:
df_subte = df_subte.rename(columns={"long": "lon","estacion":"stop_name", "id": "stop_id"})
df_subte["stop_id"] = df_subte["stop_id"].astype(int)

df_subte["type"] = "subte"

df_subte = df_subte[["linea", "stop_name", "lat", "lon", "type"]]

In [52]:
df_subte.head()

,linea,stop_name,lat,lon,type
0,H,CASEROS,-34.635750,-58.398928,subte
1,H,INCLAN - MEZQUITA AL AHMAD,-34.629376,-58.400970,subte
2,H,HUMBERTO 1°,-34.623092,-58.402323,subte
3,H,VENEZUELA,-34.615242,-58.404732,subte
4,H,ONCE - 30 DE DICIEMBRE,-34.608935,-58.406036,subte


In [53]:
df_subte.to_parquet("../data/interim/transporte/subte.parquet", index=False, engine="pyarrow")

## Premetro

In [54]:
df_premetro = pd.read_csv("../data/external/transporte_publico/premetro/estaciones-premetro.csv")

In [55]:
df_premetro = df_premetro.rename(columns={"long": "lon","nombre":"stop_name", "id": "stop_id"})
df_premetro["type"] = "premetro"
df_premetro = df_premetro[["linea", "stop_name", "lat", "lon", "type"]]

In [56]:
df_premetro.head()

,linea,stop_name,lat,lon,type
0,PREMETRO,INTENDENTE SAGUIER,-34.643757,-58.461491,premetro
1,PREMETRO,BALBASTRO (Cementerio de Flores),-34.648601,-58.456415,premetro
2,PREMETRO,ANA MARÍA JANER,-34.659499,-58.450047,premetro
3,PREMETRO,FERNANDEZ DE LA CRUZ,-34.662225,-58.446607,premetro
4,PREMETRO,PTE. ILLIA (Lacarra),-34.665368,-58.448513,premetro


In [57]:
df_premetro.to_parquet("../data/interim/transporte/premetro.parquet", index=False, engine="pyarrow")

## Transporte en CABA y La Plata

In [58]:
df = pd.concat([df_tren, df_subte, df_premetro])

In [59]:
df.to_parquet("../data/processed/transporte/transporte.parquet", index=False, engine="pyarrow")

In [60]:
df.head()

,linea,ramal,stop_name,lat,lon,type
2455,Sarmiento,Once - Moreno,Ciudadela,-34.639923,-58.541123,tren
3051,Sarmiento,Once - Moreno,Liniers,-34.638873,-58.526341,tren
3667,Sarmiento,Once - Moreno,Villa Luro,-34.636249,-58.502131,tren
4258,Sarmiento,Once - Moreno,Floresta,-34.632376,-58.480891,tren
4849,Sarmiento,Once - Moreno,Flores,-34.627850,-58.466177,tren


In [61]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        hover_data=["linea", "ramal", "stop_name"],
                        color="type",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=900,
                        width=1600)
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Hospitales

In [62]:
with open("../data/external/salud/hospitales_CABA.json") as f:
    hospitales = json.load(f)
hosp_caba = pd.DataFrame(hospitales["features"])
hosp_plata = pd.read_csv("../data/external/salud/hospitales_LaPlata.csv", delimiter=";", encoding='latin1')

In [63]:
hosp_caba["lon"] = hosp_caba["geometry"].apply(lambda x: x["coordinates"][0])
hosp_caba["lat"] = hosp_caba["geometry"].apply(lambda x: x["coordinates"][1])
hosp_caba["name"] = hosp_caba["properties"].apply(lambda x: x["NOMBRE"])

hosp_caba["province"] = str("Capital Federal")
hosp_caba = hosp_caba[["name", "lat", "lon", "province"]]

In [64]:
hosp_caba.head(2)

,name,lat,lon,province
0,HOSPITAL GENERAL DE NIÑOS PEDRO DE ELIZALDE,-34.628847,-58.377551,Capital Federal
1,HOSPITAL GENERAL DE NIÑOS RICARDO GUTIERREZ,-34.594192,-58.412070,Capital Federal


In [65]:
hosp_caba.to_parquet("../data/interim/salud/hospitales_caba.parquet", index=False, engine="pyarrow")

In [66]:
hosp_plata = hosp_plata.drop(["DIRECCION", "TELEFONO", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7"], axis=1)
hosp_plata = hosp_plata.rename(columns={"LATITUD": "lat", "LONGITUD": "lon", "CLINICAS Y HOSPITALES": "name"})
hosp_plata["province"] = str("Bs.As. G.B.A. Zona Sur")

hosp_plata["lat"] = hosp_plata["lat"].str.replace(",", ".").replace(" ", "").astype(float)
hosp_plata["lon"] = hosp_plata["lon"].str.replace(",", ".").replace(" ", "").astype(float)

hosp_plata = hosp_plata[["name", "lat", "lon", "province"]]

In [67]:
hosp_plata.head(2)

,name,lat,lon,province
0,BREAST CLINIC S.A.,-34.907611,-57.959000,Bs.As. G.B.A. Zona Sur
1,CEMPLAN,-34.918250,-57.942444,Bs.As. G.B.A. Zona Sur


In [68]:
hosp_plata.to_parquet("../data/interim/salud/hospitales_plata.parquet", index=False, engine="pyarrow")

In [69]:
df = pd.concat([hosp_caba, hosp_plata], axis=0, ignore_index=True)

In [70]:
df.to_parquet("../data/processed/salud/hospitales.parquet", index=False, engine="pyarrow")

In [71]:
df.head(2)

,name,lat,lon,province
0,HOSPITAL GENERAL DE NIÑOS PEDRO DE ELIZALDE,-34.628847,-58.377551,Capital Federal
1,HOSPITAL GENERAL DE NIÑOS RICARDO GUTIERREZ,-34.594192,-58.412070,Capital Federal


In [72]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        # hover_data=["name"],
                        color="province",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)
fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()